In [2]:
import torch
from transformers import pipeline

d:\Installed\ana\envs\kdd_task_1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
src_lang = 'fr'
src_test_string = 'Je m\'appelle Jhon'

In [4]:
# Load model for original language to english translation
src_lang_to_eng_translator = pipeline("text2text-generation", model="facebook/m2m100_418M")

eng_lang_to_src_translator= pipeline(
    task= "translation",
    model= f"Helsinki-NLP/opus-mt-en-{src_lang}")

Device set to use cpu
Device set to use cpu


In [5]:
# translate from the src lang to eng and then back to src lang
original_to_en = src_lang_to_eng_translator(src_test_string, forced_bos_token_id=src_lang_to_eng_translator.tokenizer.get_lang_id(lang='en'))
print(original_to_en[0]['generated_text'])

en_to_original = eng_lang_to_src_translator(original_to_en[0]['generated_text'], max_length = 512)
print(en_to_original[0]['translation_text'])

My name is Jhon
Mon nom est Jhon.


In [6]:
# now have to make validation pipeline
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:

# tokenize the words using the nltk functions
tokenized_reference = word_tokenize(src_test_string)
tokenized_candidate = word_tokenize(en_to_original[0]['translation_text'])

In [ ]:
bleu_score = sentence_bleu([tokenized_reference], tokenized_candidate)
print(f'blue score: {bleu_score}')

edit_distance = nltk.edit_distance(tokenized_reference, tokenized_candidate)

normalized_edit_distance = (edit_distance / max(len(tokenized_reference), len(tokenized_candidate)))
score = 1 - normalized_edit_distance

print(f'Normalized Edit Distance {normalized_edit_distance}')
print(f'Score using the normalized edit distance only: {score}')

explain_score = (1 - normalized_edit_distance + bleu_score) / 2
print(f'Explainablity Score: {explain_score}')

blue score: 1.2183324802375697e-231
Normalized Edit Distane0.8
Score using the normalized edit distance only: 0.19999999999999996
Explainablity Score: 0.09999999999999998


d:\Installed\ana\envs\kdd_task_1\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
d:\Installed\ana\envs\kdd_task_1\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
d:\Installed\ana\envs\kdd_task_1\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoo

In [9]:
print("\nMismatch Indicator:")
print("Original:        ", ' '.join(tokenized_reference))
print("Back-Translated: ", ' '.join(tokenized_candidate))

# Simple mismatch pointer
mismatch_line = []
max_len = max(len(tokenized_candidate), len(tokenized_candidate))
for i in range(max_len):
    if i >= len(tokenized_reference) or i >= len(tokenized_candidate):
        mismatch_line.append("^")
    elif tokenized_reference[i] != tokenized_candidate[i]:
        mismatch_line.append("^")
    else:
        mismatch_line.append(" ")

print("                 ", ' '.join(mismatch_line))


Mismatch Indicator:
Original:         Je m'appelle Jhon
Back-Translated:  Mon nom est Jhon .
                  ^ ^ ^ ^ ^
